In [73]:

import csv
import os
import timeit

import argparse
import h5py
import logging
import numpy as np
import ufl

from dolfinx import cpp, fem, io, mesh
from mpi4py import MPI
from petsc4py import PETSc

import commons, configs, constants

markers = commons.SurfaceMarkers()

In [74]:
data_dir = "mesh/study_2/test6/470-470-5_000-000-000/"
tetpath = os.path.join(data_dir, "tetr.h5")
tripath = os.path.join(data_dir, "tria.h5")

In [75]:
geo2d = h5py.File(tripath, "r")
geo3d = h5py.File(tetpath, "r")

In [76]:
geo2d.keys(), geo3d.keys()

(<KeysViewHDF5 ['data0', 'data1', 'data2']>,
 <KeysViewHDF5 ['data0', 'data1', 'data2']>)

In [77]:
geo2d['data0'].shape, geo2d['data1'].shape, geo2d['data2'].shape

((420670, 3), (547158, 3), (547158,))

In [78]:
geo3d['data0'].shape, geo3d['data1'].shape, geo3d['data2'].shape

((420670, 3), (1738202, 4), (1738202,))

In [79]:
current3d = h5py.File("mesh/study_2/test6/470-470-5_000-000-000/current.h5", "r")

In [80]:
values = current3d['Function']['f']['0']

In [90]:
comm = MPI.COMM_WORLD
with io.XDMFFile(comm, tetpath.replace(".h5", ".xdmf"), "r") as infile3:
    domain = infile3.read_mesh(cpp.mesh.GhostMode.none, 'Grid')
domain.topology.create_connectivity(domain.topology.dim, domain.topology.dim - 1)
with io.XDMFFile(comm, tripath.replace(".h5", ".xdmf"), "r") as infile2:
    domain2 = infile2.read_mesh(cpp.mesh.GhostMode.none, 'Grid')
    ft = infile2.read_meshtags(domain, name="Grid")
meshtags = mesh.meshtags(domain, 2, ft.indices, ft.values)

In [91]:
W = fem.FunctionSpace(domain, ("Lagrange", 1))
current = fem.Function(W)
cdf_fun = fem.Function(W)
cdf_fun2 = fem.Function(W)

In [93]:
ds = ufl.Measure('ds', domain=domain, subdomain_data=meshtags)
area_left_cc = fem.assemble_scalar(fem.form(1 * ds(markers.left_cc)))
area_right_cc = fem.assemble_scalar(fem.form(1 * ds(markers.right_cc)))

In [94]:
valz = set(np.asarray(geo2d['data1']).flatten())
for dof in valz:
    current.vector.setValueLocal(dof, values[dof][2])

In [97]:
min_cd = np.min(current.x.array)
max_cd = np.max(current.x.array)
cd_space = np.linspace(min_cd, max_cd, num=1000)
cdf_values = []
Id = ufl.Identity(3)
EPS = 1e-30


def check_condition(v1, check_value=1):
    v2 = lambda x: check_value * (x[0] + EPS) / (x[0] + EPS)
    cdf_fun.interpolate(v2)
    return ufl.conditional(ufl.le(v1, cdf_fun), 1, 0)

for v in cd_space:
    new_v = fem.Expression(check_condition(current, v), W.element.interpolation_points())
    cdf_fun2.interpolate(new_v)
    lpvalue = fem.assemble_scalar(fem.form(cdf_fun2 * ds(markers.left_cc))) / area_left_cc
    rpvalue = fem.assemble_scalar(fem.form(cdf_fun2 * ds(markers.right_cc))) / area_right_cc
    cdf_values.append({'i [A/m2]': v, "p_left": lpvalue, "p_right": rpvalue})
stats_path = os.path.join(data_dir, 'cdf.csv')
with open(stats_path, 'w') as fp:
    writer = csv.DictWriter(fp, fieldnames=['i [A/m2]', 'p_left', 'p_right'])
    writer.writeheader()
    for row in cdf_values:
        writer.writerow(row)

-4529.621434439466


INFO:root:running build_ext
INFO:root:building 'libffcx_expressions_0975ebfdd3aa4da6930f2aed2f6dd1067251a3cd' extension
INFO:root:/home/molel/.conda/envs/fenicsx-env/bin/x86_64-conda-linux-gnu-cc -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/molel/.conda/envs/fenicsx-env/include -fPIC -O2 -isystem /home/molel/.conda/envs/fenicsx-env/include -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/molel/.conda/envs/fenicsx-env/include -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/molel/.conda/envs/fenicsx-env/include -fPIC -I/home/molel/.conda/envs/fenicsx-env/lib/python3.10/site-packages/ffcx/codegeneration -I/home/molel/.conda/envs/fenicsx-env/include/python3.10 -c libffcx_expressions_0975ebfdd3aa4da6930f2aed2f6dd1067251a3cd.c -o ./libffcx_expressions_0975ebfdd3aa4da6930f2aed2f6dd1067251a3cd.o -O2 -g0
INFO:root:/home/molel/.conda/envs/fenicsx-env/bin/x86_64-conda-linux-g

-4490.825799163008


INFO:root:/home/molel/.conda/envs/fenicsx-env/bin/x86_64-conda-linux-gnu-cc -shared -Wl,--allow-shlib-undefined -Wl,-rpath,/home/molel/.conda/envs/fenicsx-env/lib -Wl,-rpath-link,/home/molel/.conda/envs/fenicsx-env/lib -L/home/molel/.conda/envs/fenicsx-env/lib -Wl,--allow-shlib-undefined -Wl,-rpath,/home/molel/.conda/envs/fenicsx-env/lib -Wl,-rpath-link,/home/molel/.conda/envs/fenicsx-env/lib -L/home/molel/.conda/envs/fenicsx-env/lib -Wl,-O2 -Wl,--sort-common -Wl,--as-needed -Wl,-z,relro -Wl,-z,now -Wl,--disable-new-dtags -Wl,--gc-sections -Wl,--allow-shlib-undefined -Wl,-rpath,/home/molel/.conda/envs/fenicsx-env/lib -Wl,-rpath-link,/home/molel/.conda/envs/fenicsx-env/lib -L/home/molel/.conda/envs/fenicsx-env/lib -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/molel/.conda/envs/fenicsx-env/include -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/molel/.conda/envs/fenicsx-env/include ./libffcx_expressions_

-4452.03016388655
-4413.234528610093
-4374.438893333635
-4335.643258057177
-4296.84762278072
-4258.051987504262
-4219.256352227804
-4180.4607169513465
-4141.66508167489
-4102.869446398432
-4064.073811121974
-4025.278175845516
-3986.4825405690585
-3947.6869052926013
-3908.8912700161436
-3870.095634739686
-3831.2999994632282
-3792.5043641867705
-3753.708728910313
-3714.913093633855
-3676.1174583573975
-3637.32182308094
-3598.526187804482
-3559.7305525280244
-3520.9349172515667
-3482.139281975109
-3443.3436466986514
-3404.5480114221937
-3365.7523761457364
-3326.9567408692787
-3288.161105592821
-3249.3654703163634
-3210.5698350399057
-3171.7741997634485
-3132.9785644869908
-3094.182929210533
-3055.3872939340754
-3016.5916586576177
-2977.79602338116
-2939.0003881047023
-2900.2047528282446
-2861.409117551787
-2822.6134822753293
-2783.8178469988716
-2745.022211722414
-2706.226576445956
-2667.4309411694985


KeyboardInterrupt: 